# 1.transformers库调用embed模型

In [18]:
#! pip install transformers
#! pip install langchain
# ! pip install sentence_transformers
# ! pip install transformers==4.38.2
! pip install transformers==4.41.1 peft==0.13.2
# ! pip install peft --upgrade

  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)
Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninstalling peft-0.15.2:
      Successfully uninstalled peft-0.15.2


In [22]:
! pip install nltk

In [4]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('iic/nlp_gte_sentence-embedding_chinese-large',local_dir="../model/nlp_gte_sentence")

2025-04-24 13:02:46,294 - modelscope - WARNING - Model revision not specified, use revision: v1.1.0
2025-04-24 13:02:46,695 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-04-24 13:04:27,996 - modelscope - INFO - Download model 'iic/nlp_gte_sentence-embedding_chinese-large' successfully.


In [5]:
from transformers import AutoTokenizer,AutoModel
from sentence_transformers.util import cos_sim
import torch.nn.functional as F

In [6]:
input_text = [
    "中国的首都在哪里？",
    "你喜欢去哪里旅游？",
    "北京",
    "今天中午吃什么?"
] 


In [8]:
model_path = "../model/nlp_gte_sentence"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path,device_map='cpu')

e:\program\anaconda\envs\llamaIndex\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
batch_tokens = tokenizer(input_text,
                         max_length=30,
                         padding=True, # 特殊的token会补齐
                         truncation=True,
                         return_tensors='pt')

In [11]:
batch_tokens[0].tokens

['[CLS]', '中', '国', '的', '首', '都', '在', '哪', '里', '？', '[SEP]']

In [12]:
batch_tokens[2].tokens

['[CLS]',
 '北',
 '京',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [13]:
outputs = model(**batch_tokens)

In [14]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 2.1403,  0.3203, -0.0490,  ...,  0.5848,  0.3857, -0.8572],
         [ 1.5582,  0.2515, -0.2202,  ...,  0.1754,  0.3884, -0.2534],
         [ 2.1936, -0.2080, -0.0116,  ...,  0.6195,  0.4255, -0.3223],
         ...,
         [ 1.6631,  0.5095, -0.1380,  ...,  0.6245,  0.5633, -0.5226],
         [ 1.6967,  0.1984, -0.0958,  ...,  0.6927,  0.4791, -0.3583],
         [ 2.1403,  0.3201, -0.0490,  ...,  0.5849,  0.3858, -0.8572]],

        [[ 0.4746, -0.8099,  0.4018,  ..., -0.1419,  0.4709, -1.1428],
         [ 0.1442, -0.4523,  0.5739,  ...,  0.4776, -0.1985, -0.5634],
         [-0.2576, -0.7512,  0.6273,  ...,  0.3786,  0.3548, -1.0328],
         ...,
         [ 0.1685, -0.8034,  0.3882,  ...,  0.2832,  0.2688, -0.7559],
         [ 0.0606, -0.6139,  0.3069,  ...,  0.1961,  0.1721, -0.6012],
         [ 0.4747, -0.8102,  0.4018,  ..., -0.1417,  0.4708, -1.1427]],

        [[ 0.6939,  0.8812, -0.5522,  ...,  1.1094, -

In [15]:
outputs.last_hidden_state.shape

torch.Size([4, 11, 1024])

In [16]:
# 输入句子的第一个分类的token作为embedding
embdedding = outputs.last_hidden_state[:,0]
embdedding.shape

torch.Size([4, 1024])

In [17]:
# 计算上面四个句子的相似度，先做归一化
embeddings = F.normalize(embdedding,p=2,dim=1)

In [18]:
for i in range(1,4):
    print(input_text[0],input_text[i],cos_sim(embeddings[0],embeddings[i]))

中国的首都在哪里？ 你喜欢去哪里旅游？ tensor([[0.3372]], grad_fn=<MmBackward0>)
中国的首都在哪里？ 北京 tensor([[0.6408]], grad_fn=<MmBackward0>)
中国的首都在哪里？ 今天中午吃什么? tensor([[0.2843]], grad_fn=<MmBackward0>)


# 2.sentence_transformers方式

Sentence—transformer是一个基于pytorch和Transformers的python库，专门用于句子、文本和图像嵌入

In [ ]:
from sentence_transformers.util import cos_sim
from sentence_transformers import SentenceTransformer

In [ ]:
input_text = [
    "中国的首都在哪里？",
    "你喜欢去哪里旅游？",
    "北京",
    "今天中午吃什么?"
] 


In [ ]:
model_path = "../model/nlp_gte_sentence"
model = SentenceTransformer(model_path)
embeddings =model.encode(input_text)
embeddings.shape

In [ ]:
for i in range(1,4):
    print(input_text[0],input_text[i],cos_sim(embeddings[0],embeddings[i]))

# 3.langchain方式对sentence-transformer封装

In [1]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from sentence_transformers.util import cos_sim

In [2]:
model_path = "../model/nlp_gte_sentence"

In [3]:
model = HuggingFaceBgeEmbeddings(model_name=model_path,
                                 model_kwargs={"device":"cpu"})

C:\Users\Yun\AppData\Local\Temp\ipykernel_53460\1355823959.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceBgeEmbeddings(model_name=model_path,
No sentence-transformers model found with name ../model/nlp_gte_sentence. Creating a new one with mean pooling.


In [5]:
input_text = [
    "中国的首都在哪里？",
    "你喜欢去哪里旅游？",
    "北京",
    "今天中午吃什么?"
] 

In [11]:
embeddings = model.embed_documents(input_text)

In [12]:
embeddings

[[1.8656220436096191,
  0.23187847435474396,
  -0.10798835754394531,
  0.7503786087036133,
  -0.01449113991111517,
  0.053094349801540375,
  0.12982024252414703,
  -1.7261356115341187,
  1.122061848640442,
  -0.8153597116470337,
  0.4083459973335266,
  0.6444839239120483,
  0.12038028985261917,
  -0.2617703080177307,
  -0.8534559607505798,
  -0.3640901446342468,
  -0.11626109480857849,
  -0.48533889651298523,
  -0.06065826117992401,
  0.9200891256332397,
  -1.1016685962677002,
  0.838887631893158,
  -0.0997118204832077,
  -0.5751625299453735,
  0.3025130331516266,
  0.7581635117530823,
  -0.05746316909790039,
  -0.5671433210372925,
  0.9725658893585205,
  1.1073309183120728,
  1.9028124809265137,
  -0.6233410835266113,
  -0.462600976228714,
  -0.2217700481414795,
  -0.31942614912986755,
  0.25627556443214417,
  0.07112137973308563,
  -0.5440868735313416,
  0.6998050808906555,
  -0.1210559532046318,
  0.2301454395055771,
  -0.8088634610176086,
  -0.0712364986538887,
  0.3774196803569793

In [13]:
import numpy as np

embeddings = np.array(embeddings)
print(embeddings.shape)

(4, 1024)


In [14]:
for i in range(1,4):
    print(input_text[0],input_text[i],cos_sim(embeddings[0],embeddings[i]))

中国的首都在哪里？ 你喜欢去哪里旅游？ tensor([[0.3339]], dtype=torch.float64)
中国的首都在哪里？ 北京 tensor([[0.6130]], dtype=torch.float64)
中国的首都在哪里？ 今天中午吃什么? tensor([[0.2840]], dtype=torch.float64)


# 4.embedding操作（距离计算和聚类）

## 距离计算

In [15]:
# 余弦相似度
a = embeddings[0]
b = embeddings[2]

In [16]:
from numpy import dot
from numpy.linalg import norm

In [17]:
cos_a_b = dot(a,b)/(norm(a)*norm(b))

In [18]:
print(cos_a_b,cos_sim(a,b))

0.6129714166176911 tensor([[0.6130]], dtype=torch.float64)


In [19]:
# 欧几里得距离
norm(a-b)

20.437773237762055

## 聚类计算

In [20]:
text = ["苹果","菠萝","西瓜","斑马","大象","老鼠"] 

In [21]:
output_embeddings = model.embed_documents(text)

In [22]:
from sklearn.cluster import KMeans

In [29]:
kmeans = KMeans(n_clusters=2)

In [30]:
kmeans.fit(output_embeddings)

KMeans(n_clusters=2)

In [31]:
label = kmeans.labels_

In [32]:
for i in range(len(text)):
    print(f"cls({text[i]})={label[i]}")

cls(苹果)=0
cls(菠萝)=0
cls(西瓜)=0
cls(斑马)=1
cls(大象)=1
cls(老鼠)=1
